In [1]:
from langchain_groq import ChatGroq

In [2]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [10]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://boards.greenhouse.io/spacex/jobs/7514164002?gh_jid=7514164002")
page_data = loader.load().pop().page_content
print(page_data)






Job Application for Backend Software Engineer (Starlink) at SpaceX























      Apply Now
    
Backend Software Engineer (Starlink)

    at SpaceX
  

      Redmond, WA
    


SpaceX was founded under the belief that a future where humanity is out exploring the stars is fundamentally more exciting than one where we are not. Today SpaceX is actively developing the technologies to make this possible, with the ultimate goal of enabling human life on Mars.
BACKEND SOFTWARE ENGINEER (STARLINK)
At SpaceX, we’re leveraging our experience in building rockets and spacecraft to deploy Starlink, the world’s most advanced broadband internet system. Starlink is the world’s largest satellite constellation and is providing fast, reliable internet to 3M+ users worldwide. We design, build, test, and operate all parts of the system – thousands of satellites, consumer receivers that allow users to connect within minutes of unboxing, and the software that brings it all together. We’ve

In [11]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [12]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Backend Software Engineer (Starlink)',
 'experience': '1+ years of professional experience in software development; OR 3+ years of professional experience in software development in lieu of a degree (internship experience is applicable)',
 'skills': ['Python',
  'Relational databases (PostgreSQL, CockroachDB, or similar)',
  'Systems-level design of highly-critical software infrastructure',
  'C++',
  'Golang',
  'Distributed systems or large-scale consumer internet systems',
  'Containerization technologies (e.g. Docker, Kubernetes)',
  'Version control, testing, continuous integration, build, deployment, and monitoring',
  'Profiling and improving application performance'],
 'description': 'As a software engineer working in Starlink’s GNC (Guidance, Navigation, and Control) organization, you will use your distributed systems knowledge to build mission-critical systems that keep low-earth orbit safe. You’ll work on collision avoidance, maneuver coordination, and orbital pred

In [13]:
type(json_res)

dict

In [14]:
import pandas as pd

df = pd.read_csv("app/resource/my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [15]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [19]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/android-portfolio'}]]

In [20]:
job

{'role': 'Backend Software Engineer (Starlink)',
 'experience': '1+ years of professional experience in software development; OR 3+ years of professional experience in software development in lieu of a degree (internship experience is applicable)',
 'skills': ['Python',
  'Relational databases (PostgreSQL, CockroachDB, or similar)',
  'Systems-level design of highly-critical software infrastructure',
  'C++',
  'Golang',
  'Distributed systems or large-scale consumer internet systems',
  'Containerization technologies (e.g. Docker, Kubernetes)',
  'Version control, testing, continuous integration, build, deployment, and monitoring',
  'Profiling and improving application performance'],
 'description': 'As a software engineer working in Starlink’s GNC (Guidance, Navigation, and Control) organization, you will use your distributed systems knowledge to build mission-critical systems that keep low-earth orbit safe. You’ll work on collision avoidance, maneuver coordination, and orbital pred

In [18]:
job = json_res
job['skills']

['Python',
 'Relational databases (PostgreSQL, CockroachDB, or similar)',
 'Systems-level design of highly-critical software infrastructure',
 'C++',
 'Golang',
 'Distributed systems or large-scale consumer internet systems',
 'Containerization technologies (e.g. Docker, Kubernetes)',
 'Version control, testing, continuous integration, build, deployment, and monitoring',
 'Profiling and improving application performance']

In [22]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Alif, a business development executive at DataInc. DataInc is an AI & Software company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase DataInc's portfolio: {link_list}
        Remember you are Alif, BDE at DataInc. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Backend Software Engineering Services for Starlink's GNC Organization

Dear Hiring Manager,

I came across the job posting for a Backend Software Engineer (Starlink) and was impressed by the cutting-edge work being done in the Guidance, Navigation, and Control organization. As a Business Development Executive at DataInc, I'd like to introduce our team's expertise in delivering high-quality software solutions that align with your requirements.

Our team at DataInc has extensive experience in developing scalable and efficient software systems using Python, C++, and Golang. We've worked on various projects involving relational databases (PostgreSQL, CockroachDB), containerization technologies (Docker, Kubernetes), and distributed systems. Our expertise in systems-level design, version control, testing, continuous integration, build, deployment, and monitoring will ensure seamless integration with your existing infrastructure.

I'd like to highlight a few relevant projects 